This notebook is used to reproduce the Figure 6 in the paper.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ortools_solver import CVRP_solver
from utils import random_task_generation
from pipette_scheduler import calculate_D, calculate_S_E, calculate_D_prime
import pandas as pd

In [ ]:
np.random.seed(0)
num_candidates = 2000
a = random_task_generation(96,96,num_candidates)
D_S = calculate_D(a.shape[0])
D_D = calculate_D(a.shape[1])
S, E, volumes = calculate_S_E(a)
# calculate distance matrix
D_prime = calculate_D_prime(D_S, D_D, S, E, volumes, 1, 100, 1, 100)
# VRP solver
VRP_distance, VRP_recorder = CVRP_solver((D_prime * 100).astype(np.int64), solving_time=20)


In [3]:
stats = []
np.random.seed(0)
for r in range(6):
    num_candidates = 2000
    # generate random tasks
    a = random_task_generation(96,96,num_candidates)
    D_S = calculate_D(a.shape[0])
    D_D = calculate_D(a.shape[1])
    S, E, volumes = calculate_S_E(a)
    D_prime = calculate_D_prime(D_S, D_D, S, E, volumes, 1, 100, 1, 100)
    # do a scanning of solving time
    for i in range(2,71,2):
        # VRP solver
        CVRP_distance, CVRP_recorder = CVRP_solver(np.round(D_prime*100).astype(np.int64), solving_time=i)
        CVRP_distance = CVRP_distance / 100
        print(f'CVRP_distance: {CVRP_distance}')
        stats.append([r,i, VRP_distance])

VRP_distance: 3374.0
VRP_distance: 3374.0
VRP_distance: 3350.9
VRP_distance: 3314.44
VRP_distance: 3295.74
VRP_distance: 3276.08
VRP_distance: 3265.32
VRP_distance: 3256.35
VRP_distance: 3250.58
VRP_distance: 3247.86
VRP_distance: 3245.66
VRP_distance: 3243.93
VRP_distance: 3235.63
VRP_distance: 3224.65
VRP_distance: 3213.41
VRP_distance: 3209.33
VRP_distance: 3209.33
VRP_distance: 3209.33
VRP_distance: 3209.33
VRP_distance: 3207.82
VRP_distance: 3206.0
VRP_distance: 3203.75
VRP_distance: 3201.08
VRP_distance: 3198.87
VRP_distance: 3198.87
VRP_distance: 3198.87
VRP_distance: 3196.96
VRP_distance: 3196.06
VRP_distance: 3196.06
VRP_distance: 3196.06
VRP_distance: 3196.06
VRP_distance: 3196.06
VRP_distance: 3195.97
VRP_distance: 3195.97
VRP_distance: 3195.97
VRP_distance: 3420.14
VRP_distance: 3420.14
VRP_distance: 3392.66
VRP_distance: 3364.28
VRP_distance: 3340.49
VRP_distance: 3325.95
VRP_distance: 3307.7
VRP_distance: 3298.94
VRP_distance: 3294.75
VRP_distance: 3293.36
VRP_distance: 3

In [ ]:
stats_df = pd.DataFrame(stats, columns=['repeats','time','CVRP_distance'])
stats_df_grouped = stats_df.groupby('time').agg({'CVRP_distance': ['mean', 'std']}).reset_index()
# plot the results
plt.errorbar(stats_df_grouped['time'], stats_df_grouped[('CVRP_distance', 'mean')], yerr=stats_df_grouped[('CVRP_distance', 'std')], fmt='o-', capsize=5, color='#d62728')
plt.xlabel('Solution time budget (s)', fontsize=16)
#plt.title('Number of jobs = 2000', fontsize=16)
plt.ylabel('Computed execution time', fontsize=16)
# tick size
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)